In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("stabilityai/StableBeluga-7B", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/StableBeluga-7B",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
system_prompt = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.\n\n"

message = "Write me a poem please"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(
    **inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

### System:
You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.

### User: Write me a poem please

### Assistant:
 The melancholy moon gently lights the night,
As a whispy breeze softly plays its flute,
Creating the perfect atmosphere for a romantic night,
Wrapped in warm, deep hugs, two souls in love entwine,
Laughter spills forth, sharing hearts, memories divine.

Unicorns and rainbows, imagination's delight,
Embraced by love, their hearts in perfect delight,
Harmoniously, they sway together, blissful and free,
As the moon, stars, and heart-shaped chocolate, sigh in ecstasy.


In [9]:
system_prompt = """### System:\nInstruction:

**Prompt:** Extract filters from the query and return the result in a structured JSON format. The input query will consist of two parts: a primary query and a filter. The filter will typically be in the format "from [start_date] to [end_date]" or some variation of it. Your task is to:

1. Identify and extract the primary query, which represents the main search topic.
2. Extract the start date and end date from the filter, if present.

Please make sure the JSON format for the response adheres to the following structure:

```
{
  "query": "Primary Query",
  "start_date": "Start Date (YYYY-MM-DD)",
  "end_date": "End Date (YYYY-MM-DD)"
}
```

**Example Usage:** "Extract filters from 'CCTV from 3 Oct 2017 to 3 Oct 2019'."

**Response:** 
```
{
  "query": "CCTV",
  "start_date": "2017-10-03",
  "end_date": "2019-10-03"
}
```
\n\n"""
message = "Get all documents that consists of Laptop from 2017 to 2019"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
prompt_tokens = tokenizer(prompt, return_tensors="pt").to("cuda")["input_ids"]
start_index = prompt_tokens.shape[-1]

output = model.generate(
    prompt_tokens, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256
)

generation_output = output[0][start_index:]
generation_text = tokenizer.decode(generation_output, skip_special_tokens=True)
print(generation_text)

{
  "query": "Laptop",
  "start_date": "2017-01-01",
  "end_date": "2019-12-31"
}


In [11]:
import json

data = json.loads(generation_text)

print(data)

{'query': 'Laptop', 'start_date': '2017-01-01', 'end_date': '2019-12-31'}
